In [15]:
import mediapipe as mp
print(mp.__version__)
print(hasattr(mp, "solutions"))



0.10.5
True


In [16]:
import mediapipe as mp
print(mp.__version__)
print(dir(mp))


0.10.5
['CalculatorGraph', 'GraphInputStreamAddMode', 'Image', 'ImageFormat', 'ImageFrame', 'Matrix', 'Packet', 'Timestamp', 'ValidatedGraphConfig', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'calculators', 'packet_creator', 'packet_getter', 'resource_util', 'solutions', 'tasks']


In [17]:
import cv2
import mediapipe as mp
import numpy as np

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

holis = mp_holistic.Holistic()


In [18]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe setup
holistic = mp.solutions.holistic
hands = mp.solutions.hands
drawing = mp.solutions.drawing_utils
holis = holistic.Holistic()

# DATA STORAGE (THIS WAS MISSING)
X = []
data_size = 0

cap = cv2.VideoCapture(0)

name = input("enter the name of the data: ")

while True:
    lst = []

    ret, frm = cap.read()
    if not ret:
        break

    frm = cv2.flip(frm, 1)
    res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

    # ---------- FACE ----------
    if res.face_landmarks:
        for i in res.face_landmarks.landmark:
            lst.append(i.x - res.face_landmarks.landmark[1].x)
            lst.append(i.y - res.face_landmarks.landmark[1].y)
    else:
        # 468 landmarks * 2
        for _ in range(936):
            lst.append(0.0)

    # ---------- LEFT HAND ----------
    if res.left_hand_landmarks:
        for i in res.left_hand_landmarks.landmark:
            lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
    else:
        for _ in range(42):
            lst.append(0.0)

    # ---------- RIGHT HAND ----------
    if res.right_hand_landmarks:
        for i in res.right_hand_landmarks.landmark:
            lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
    else:
        for _ in range(42):
            lst.append(0.0)

    # SAVE SAMPLE
    X.append(lst)
    data_size += 1

    # DRAW LANDMARKS
    if res.face_landmarks:
        drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_CONTOURS)
    if res.left_hand_landmarks:
        drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
    if res.right_hand_landmarks:
        drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)

    cv2.putText(frm, f"Samples: {data_size}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("window", frm)

    if cv2.waitKey(1) == 27 or data_size >= 100:
        break

cap.release()
cv2.destroyAllWindows()

# SAVE FILE
np.save(f"{name}.npy", np.array(X))
print("Saved:", name + ".npy")
print("Shape:", np.array(X).shape)


enter the name of the data:  sad


Saved: sad.npy
Shape: (100, 1020)


In [19]:
np.save(f"{name}.npy", np.array(X))   #we are storing one particular emoji for 100 samples
print(np.array(X).shape)

(100, 1020)
